In [ ]:
"""
 - get probability matrix
 - calculate randomwalk, wasserstein, symmetric KL, jaccard sp
 - plot results along with jaccard-shortest
"""

In [77]:
from scipy.io import loadmat
from scipy.sparse import find
from scipy.sparse.csgraph import dijkstra
from scipy.linalg import sqrtm
from scipy.cluster.hierarchy import linkage, dendrogram, cut_tree
from scipy.spatial.distance import pdist, squareform
from scipy.stats import wasserstein_distance, entropy
from sklearn.metrics import adjusted_rand_score
import numpy as np
import numpy.linalg as lalg
import matplotlib.pyplot as plt
import networkx as nx
from networkx.generators.community import LFR_benchmark_graph
from pickle import dump, load, HIGHEST_PROTOCOL

%matplotlib inline

In [95]:
def wrangle_karate():
    mat = loadmat('karate.mat')
    S = mat['Problem'][0][0][2]
    G = nx.Graph(S)

    mr_hi = set([1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 17, 18, 20, 22])
    split = {}
    cols = {}
    for i in G.nodes:
        split[i] = 0 if i+1 in mr_hi else 1
        cols[i] = 'red' if i+1 in mr_hi else 'black'
    nx.set_node_attributes(G, split, 'cluster')
    nx.set_node_attributes(G, cols, 'color')
    return G

In [96]:
def wrangle_football():
    # data fixed by Tim Evans
    mat = loadmat('football_fixed.mat')
    I,J = mat['links']
    G = nx.Graph()
    G.add_edges_from(zip(I,J))

    confs = {}
    cols = {}
    cmap = plt.get_cmap('tab20')
    for node in mat['nodes']:
        idx = int(node[0])
        conf = int(node[2])
        assert idx not in confs.keys()
        if conf>10:
            confs[idx] = conf
            cols[idx] = 'black'
        else:
            confs[idx] = conf
            cols[idx] = cmap(conf/11)
    nx.set_node_attributes(G, confs, 'cluster')
    nx.set_node_attributes(G, cols, 'color')
    return G

In [97]:
def wrangle_caltech():
    mat = loadmat('Caltech36.mat')
    G = nx.Graph(mat['A'])

    houses = {}
    cols = {}
    cmap = plt.get_cmap('tab10')
    for i,x in enumerate(mat['local_info']):
        house = x[4]
        if house==0:
            houses[i] = house
            cols[i] = 'black'
        else:
            houses[i] = house
            cols[i] = cmap((house-165)/10)
    nx.set_node_attributes(G, houses, 'cluster')
    nx.set_node_attributes(G, cols, 'color')
    return G

In [116]:
def generate_lfr():
    G = LFR_benchmark_graph(n=250, tau1=3, tau2=1.5, mu=.1, average_degree=5, min_community=20, seed=10)
    communities = nx.get_node_attributes(G, 'community')
    seen = set()
    clust_idx = 0
    clusts = {}
    cols = {}
    cmap = plt.get_cmap('tab10')
    for idx,community in communities.items():
        if idx not in seen:
            for member in community:
                clusts[member] = clust_idx
                cols[member] = cmap(clust_idx/10)
                seen.add(member)
            clust_idx += 1
    nx.set_node_attributes(G, clusts, 'cluster')
    nx.set_node_attributes(G, cols, 'color')
    return G

In [112]:
def largest_connected_component(G):
    G = G.subgraph(max(nx.connected_components(G), key=len))
    G = nx.relabel.convert_node_labels_to_integers(G, label_attribute='orig')
    return G

In [113]:
def is_metric(dist, eps=0):
    square = squareform(dist)
    n,m = square.shape
    assert n==m
    for i in range(n):
        for j in range(n):
            for k in range(n):
                if square[i,j] > square[i,k]+square[k,j]+eps:
                    print(f'{i} {j} {k}')
                    print(square[i,j])
                    print(square[i,k]+square[k,j])
                    return False
    return True

def draw(hierarchy, G):
    plt.figure(figsize=(20,5))
    plt.rcParams["font.weight"] = "bold"
    dend = dendrogram(hierarchy, leaf_rotation=90, leaf_font_size=8)

    ticks, labels = plt.xticks()
    for label in labels:
        idx = int(label.get_text())
        label.set_color(G.nodes[idx]['color'])
    plt.xticks(ticks, labels)

def evaluate_ARI(hierarchy, G):
    """calculates the adjusted rand index to evaluate cluster quality"""
    attrs = nx.get_node_attributes(G, 'cluster')
    n = len(G)
    gtruth = []
    ignore = set() # unuseful labels are set to -1 earlier
    for idx,label in attrs.items():
        if label >= 0:
            gtruth.append(label)
        else:
            ignore.add(idx)
    labelset = set(gtruth)
    predicted = []
    
    # try cutting at every dendrogram branch
    cut = cut_tree(hierarchy).transpose()
    best = -float('inf')
    argbest = -1
    for n_clust, clusters in enumerate(cut):
        score = adjusted_rand_score(gtruth, clusters)
        if score > best:
            best = score
            argbest = n_clust
    return best, n-argbest

In [84]:
def markov_matrix(G):
    '''gets the markov matrix and diagonal degrees matrix (used again for walktrap) from a graph G'''
    n = len(G)
    A = nx.to_numpy_array(G)

    for i in range(n):
        for j in range(i+1,n):
            assert A[i,j]==A[j,i]

    D = np.identity(n) * np.sum(A, axis=1)
    P = lalg.inv(D) @ A
    return P,D

def embed_snapshot_markov(markov, degrees, steps=4):
    '''embedding of pons & lapaty'''
    return lalg.inv(sqrtm(degrees)) @ lalg.matrix_power(markov,steps)

def embed_damped_markov(markov, damping, steps=100):
    '''weights each markov chain state according to a damping factor'''
    state = np.array(markov)
    final = np.array(markov)
    totaldamp = 1
    currdamp = damping
    for i in range(1,steps):
        state = state @ markov
        final += currdamp * state
        totaldamp += currdamp
        currdamp *= damping
    final /= totaldamp
    return final

In [85]:
# different clusterings

def eval_cluster_walktrap(embedding):
    dist = pdist(embedding, metric='euclidean')
    # print(is_metric(dist))
    Z = linkage(dist, method='ward', optimal_ordering=False)
    # draw(Z, G)
    return evaluate_ARI(Z, G)

def eval_KL_symmetric(embedding):
    if min(np.nditer(embedding)) <= 0:
        return (-float('inf'), 0)
    dist = pdist(embedding, metric=lambda x,y: entropy(x,y)+entropy(y,x))
    # print(is_metric(dist))
    Z = linkage(dist, method='ward', optimal_ordering=False)
    # draw(Z, G)
    return evaluate_ARI(Z, G)

def eval_wasserstein(embedding):
    dist = pdist(embedding, wasserstein_distance)
    # print(is_metric(squareform(dist), 1e-10))
    Z = linkage(dist, method='ward', optimal_ordering=False)
    # draw(Z, G)
    return evaluate_ARI(Z, G)

def eval_jaccard_sp(G):
    A = nx.to_numpy_array(G)
    jacc = pdist(A, metric='jaccard')
    bacon = dijkstra(A, directed=False, unweighted=True)
    dist = jacc * squareform(bacon)

    # print(is_metric(dist))
    Z = linkage(dist, method='ward', optimal_ordering=False)
    # draw(Z, G)
    return evaluate_ARI(Z, G)

def eval_commute(G):
    # first get unnormalised laplacian
    n = len(G)
    A = nx.to_numpy_array(G)
    D = np.identity(n) * np.sum(A, axis=1)
    L = D-A
    
    # get moore-penrose pseudo-inverse
    Lplus = lalg.pinv(L)
    vol = 2*len(G.edges)
    
    # use to calculate ECT
    def commute(i,j):
        return vol * (Lplus[i,i] + Lplus[j,j] - 2*Lplus[i,j])
    
    dist = []
    for i in range(n):
        for j in range(i+1,n):
            dist.append(commute(i,j))
#     print(is_metric(dist, 1e-10))
    Z = linkage(dist, method='ward', optimal_ordering=False)
#     draw(Z, G)
    return evaluate_ARI(Z, G)

In [86]:
def do_experiment(filename, G):
    G = largest_connected_component(G)
    # jaccard * shortestpaths
    results = {}
    results['jaccsp'] = eval_jaccard_sp(G)

    # euclidean commute time
    results['commute'] = eval_commute(G)

    # prepare markov chain for embedding
    markov, degrees = markov_matrix(G)
    identity = np.identity(len(G))

    # walktrap, KL, wasserstein (all with both types of embedding)
    all_steps = [1,2,3,4,5,6,7,8,9,10,100]
    for steps in all_steps:
        embedding = embed_snapshot_markov(markov, degrees, steps)
        results[f'snapshotD_walktrap_{steps:03d}'] = eval_cluster_walktrap(embedding)
        results[f'snapshotD_KL_{steps:03d}'] = eval_KL_symmetric(embedding)
        results[f'snapshotD_wasserstein_{steps:03d}'] = eval_wasserstein(embedding)

        embedding = embed_snapshot_markov(markov, identity, steps)
        results[f'snapshotI_walktrap_{steps:03d}'] = eval_cluster_walktrap(embedding)
        results[f'snapshotI_KL_{steps:03d}'] = eval_KL_symmetric(embedding)
        results[f'snapshotI_wasserstein_{steps:03d}'] = eval_wasserstein(embedding)

    # damped markov embedding
    all_damps = [.1,.2,.3,.4,.5,.6,.7,.8,.85,.9,1]
    for damp in all_damps:
        embedding = embed_damped_markov(markov, damp)
        dampstr = str(damp).replace('.','p')
        results[f'damped_walktrap_{dampstr}'] = eval_cluster_walktrap(embedding)
        results[f'damped_KL_{dampstr}'] = eval_KL_symmetric(embedding)
        results[f'damped_wasserstein_{dampstr}'] = eval_wasserstein(embedding)

    with open(f'{filename}.pkl', 'wb') as f: 
        dump(results, f, HIGHEST_PROTOCOL)

In [119]:
# do_experiment('karate', wrangle_karate())
# do_experiment('football', wrangle_football())
# do_experiment('caltech', wrangle_caltech())
# do_experiment('lfr', wrangle_lfr())

G = generate_lfr()
eval_commute(G)

(0.8288552927444437, 5)

In [22]:
with open(f'caltech.pkl', 'rb') as f:
    results = load(f)
    for label, result in sorted(results.items()):
        print(f"{label:<25} {result[0]:.3f} {result[1]}")

commute                   0.000 2
damped_KL_0p1             0.351 12
damped_KL_0p2             0.380 20
damped_KL_0p3             0.395 19
damped_KL_0p4             0.396 19
damped_KL_0p5             0.387 19
damped_KL_0p6             0.373 27
damped_KL_0p7             0.397 24
damped_KL_0p8             0.410 28
damped_KL_0p85            0.400 29
damped_KL_0p9             0.376 28
damped_KL_1               0.365 59
damped_walktrap_0p1       0.404 78
damped_walktrap_0p2       0.360 99
damped_walktrap_0p3       0.371 73
damped_walktrap_0p4       0.377 98
damped_walktrap_0p5       0.383 64
damped_walktrap_0p6       0.394 95
damped_walktrap_0p7       0.406 65
damped_walktrap_0p8       0.397 74
damped_walktrap_0p85      0.417 56
damped_walktrap_0p9       0.432 52
damped_walktrap_1         0.386 67
damped_wasserstein_0p1    0.038 2
damped_wasserstein_0p2    0.038 2
damped_wasserstein_0p3    0.037 2
damped_wasserstein_0p4    0.037 2
damped_wasserstein_0p5    0.032 2
damped_wasserstein_0p6    